# Fragility Analyses With Row/Column Perturbations

Since the linear operator estimated from the windowed data matrices are not symmetric, then the corresponding row and column perturbations are different. Although there lacks some theory to describe how different they are depending on matrix properties, we can empirically investigate the differences in the context of EZ localization.

Here, we:

1. load and visualize the column/row perturbed fragility heaatmaps
2. load and visualize their combinations (mean, ratio)

In [1]:
import numpy as np
import pandas as pd
import mne
import os
import json
import os.path as op
from pathlib import Path
import collections
from natsort import natsorted

from sklearn.datasets import make_spd_matrix
from mne.io import RawArray
from mne import create_info
from mne_bids import BIDSPath, get_entity_vals, read_raw_bids
import mne
mne.utils.use_log_level('error')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl

import dabest

from eztrack.io.base import _add_desc_to_bids_fname, concatenate_derivs
from eztrack.io import read_derivative_npy, DerivativeArray, create_deriv_info
from eztrack.fragility import (lds_raw_fragility, DiscreteLinearSystem, 
                               SystemIDModel, StructuredPerturbationModel)
from eztrack.viz import _load_turbo, generate_heatmap
from eztrack.posthoc.hypo import compute_null

_load_turbo()

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def get_resected_chs(subject, root):
    bids_path = BIDSPath(subject=subject, root=root,
                         suffix='channels', extension='.tsv')
    ch_fpaths = bids_path.match()

    # read in sidecar channels.tsv
    channels_pd = pd.read_csv(ch_fpaths[0], sep='\t')
    description_chs = pd.Series(channels_pd.description.values, index=channels_pd.name).to_dict()
    resected_chs = [ch for ch, description in description_chs.items() if description == 'resected']
    return resected_chs

# Configure Paths and BIDS related metadata

In [77]:
# load in originally ran npy data
root = Path('/Users/adam2392/OneDrive - Johns Hopkins/sickkids/')
# root = Path('/Users/adam2392/Dropbox/epilepsy_bids/')
deriv_root = root / 'derivatives'
reference = 'monopolar'
sampling_res = '1000Hz'  # 'originalsampling'
deriv_chain = Path(sampling_res) / 'radius1.25' / 'fragility' / reference
deriv_path = deriv_root / deriv_chain

figures_path = deriv_root / 'figures'

sessions = ['extraoperative', 'preresection', 
            'intraresection', 'postresection']
desc = 'perturbmatrix'
rowdesc = 'rowperturbmatrix'

In [78]:
subject = 'E1'
session = 'extraoperative'
task = 'ictal'
acquisition = 'ecog'
run = '01'
datatype = 'ieeg'
extension = '.json'

# subject = 'pt1'
# session = 'presurgery'
# acquisition = 'ecog'
# run = '01'

In [79]:
subjects = get_entity_vals(deriv_path, 'subject')
ignore_subjects = [sub for sub in subjects if sub != subject]

sessions = get_entity_vals(deriv_path, 'session', ignore_subjects=ignore_subjects)
ignore_sessions = [ses for ses in sessions if ses != session]

tasks = get_entity_vals(deriv_path, 'task', 
                        ignore_subjects=ignore_subjects,
                        ignore_sessions=ignore_sessions
                       )

print(f'These are the tasks for {subject} {session}: {tasks}')
task = tasks[-1]

print(task)

These are the tasks for E1 extraoperative: ['ictal', 'interictalasleep', 'interictalawake']
interictalawake


In [80]:
bids_path = BIDSPath(subject=subject, session=session,task=task,
                    acquisition=acquisition, run=run, suffix=datatype,
                    datatype=datatype, extension=extension,
                    root=root)
print(bids_path)

/Users/adam2392/OneDrive - Johns Hopkins/sickkids/sub-E1/ses-extraoperative/ieeg/sub-E1_ses-extraoperative_task-interictalawake_acq-ecog_run-01_ieeg.json


In [81]:
deriv_fname = _add_desc_to_bids_fname(bids_path.basename, description=desc)
print(deriv_fname)

sub-E1_ses-extraoperative_task-interictalawake_acq-ecog_run-01_desc-perturbmatrix_ieeg.json


In [82]:
deriv_fpath = deriv_root / deriv_chain / f'sub-{subject}' / deriv_fname
print(deriv_fpath)
print(deriv_fpath.exists())

/Users/adam2392/OneDrive - Johns Hopkins/sickkids/derivatives/1000Hz/radius1.25/fragility/monopolar/sub-E1/sub-E1_ses-extraoperative_task-interictalawake_acq-ecog_run-01_desc-perturbmatrix_ieeg.json
True
